In [1]:
import scvelo as scv
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import cellrank as cr
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

import psutil

mem = psutil.virtual_memory()
print(f"Total memory: {mem.total / 1e9:.2f} GB")
print(f"Available memory: {mem.available / 1e9:.2f} GB")
print(f"Used memory: {mem.used / 1e9:.2f} GB")
print(f"Memory usage: {mem.percent}%")
# print(globals().keys())

import gc
gc.collect()


Total memory: 405.41 GB
Available memory: 336.06 GB
Used memory: 65.61 GB
Memory usage: 17.1%


0

In [18]:
# org_merge_changed_D60 = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_changed_D60.h5ad")
# org_merge_changed_D100 = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_changed_D100.h5ad")
# org_merge_D100_KO = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_D100_KO.h5ad")

# org_merge_by_D60 = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_by_D60.h5ad")
org_merge_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_changed.h5ad")

In [19]:
print(org_merge_changed)
# scv.pl.scatter(org_merge_by_D60, basis="X_umap.css.sigPCs", color="HNOCA_annot_level_2_pruned")

AnnData object with n_obs × n_vars = 94531 × 1922
    obs: 'nCount_ambiguous', 'nFeature_ambiguous', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'RNA_snn_res.0.1', 'seurat_clusters', 'pANN_0.25_0.3_354', 'doublet_finder', 'pANN_0.25_0.03_363', 'pANN_0.25_0.005_663', 'pANN_0.25_0.17_698', 'pANN_0.25_0.005_717', 'pANN_0.25_0.01_706', 'pANN_0.25_0.005_755', 'pANN_0.25_0.001_663', 'pANN_0.25_0.26_795', 'Sample_ID', 'unintegrated_clusters', 'clusters_CSS_allEmbeddingDims_Point1', 'clusters_CSS_SigPs_Point1', 'clusters_Harmony_Point1', 'clusters_CSS_allEmbeddingDims_Point2', 'clusters_CSS_SigPs_Point2', 'clusters_Harmony_Point2', 'clusters_CSS_allEmbeddingDims_Point5', 'clusters_CSS_SigPs_Point5', 'clusters_Harmony_Point5', 'clusters_CSS_SigPs_0.6', 'Genotype', 'Day', 'HNOCA_annot_level_1_pruned', 'HNOCA_annot_level_2_pruned', 'css_cluster', 'RNA_snn_res.0.8', 'initial_

In [4]:
scv.pp.filter_and_normalize(org_merge_by_D60, min_cells = 2, min_cells_u = 2)
scv.pp.filter_genes_dispersion(org_merge_by_D60)
sc.pp.neighbors(org_merge_by_D60, n_neighbors=20, use_rep='X_css')
scv.pp.moments(org_merge_by_D60, n_pcs=None, n_neighbors=None)

Filtered out 3317 genes that are detected in less than 2 cells (spliced).
Filtered out 2878 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


Extracted 2528 highly variable genes.
computing moments based on connectivities
    finished (0:00:08) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [5]:
scv.tl.recover_dynamics(org_merge_by_D60)

recovering dynamics (using 1/48 cores)
or disable the progress bar using `show_progress_bar=False`.
    finished (0:27:15) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)


In [6]:
scv.tl.velocity(org_merge_by_D60, mode='dynamical', diff_kinetics=True)
scv.tl.velocity_graph(org_merge_by_D60)

computing velocities
    finished (0:00:19) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/48 cores)
    finished (0:00:47) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [20]:
import pandas as pd

celltype_map = {
    "Neuroepithelium": "Neuroepithelium",
    "Dorsal Telencephalic IP": "Dorsal Telencephalic IP",
    "Dorsal Telencephalic Neuron": "Dorsal Telencephalic Neuron",
    "Ventral Telencephalic NPC": "Ventral Telencephalic NPC",
    "Ventral Telencephalic Neuron": "Ventral Telencephalic Neuron",
    "Non-telencephalic NPC": "Non-telencephalic NPC",
    "Non-telencephalic Neuron": "Non-telencephalic Neuron"
}

org_merge_changed.obs["HNOCA_annot_level_2_pruned_simplified"] = (
    org_merge_changed.obs["HNOCA_annot_level_2_pruned"] 
    .map(celltype_map)                      
    .fillna("Others")                       
)

celltype_counts = org_merge_changed.obs["HNOCA_annot_level_2_pruned_simplified"].value_counts()

print("Number of cells per simplified cell type:")
print(celltype_counts)

Number of cells per simplified cell type:
HNOCA_annot_level_2_pruned_simplified
Ventral Telencephalic Neuron    36967
Non-telencephalic Neuron        29077
Others                          10683
Neuroepithelium                 10126
Non-telencephalic NPC            3930
Ventral Telencephalic NPC        3011
Dorsal Telencephalic Neuron       452
Dorsal Telencephalic IP           285
Name: count, dtype: int64


In [8]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_by_D60_velocity_diff_kinetics_genotype_celltype_simplified_stream.pdf"

r_palette = {
    "Neuroepithelium": "#f44336",
    "Dorsal Telencephalic IP": "#8bc34a",
    "Dorsal Telencephalic Neuron": "#009688",
    "Ventral Telencephalic NPC": "#1565c0",
    "Ventral Telencephalic Neuron": "#ffc107",
    "Non-telencephalic NPC": "#ff9800",
    "Non-telencephalic Neuron": "#448aff",
    "Others": "#ad1457"
}

with PdfPages(pdf_path) as pdf:

    scv.pl.velocity_embedding_grid(
        org_merge_by_D60,
        basis='umap.css.sigPCs',
        color='HNOCA_annot_level_2_pruned_simplified',
        palette=r_palette,
        size=50,
        scale=0.3,               
        alpha=0.5,
        legend_loc='lower left',
        linewidth=1.2,
        legend_fontsize=5,
        show=False
    )

    
    fig = plt.gcf()
    fig.set_size_inches(11, 9)  
    pdf.savefig(fig)           
    plt.close()

computing velocity embedding
    finished (0:00:05) --> added
    'velocity_umap.css.sigPCs', embedded velocity vectors (adata.obsm)


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:65: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return cat(data)  # if data is categorical/array
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype

In [26]:
org_merge_changed.obs['condition_celltype'] = (
    org_merge_changed.obs['Genotype'].astype(str) + '_' +
    org_merge_changed.obs['HNOCA_annot_level_2_pruned'].astype(str)
).astype('category')
scv.tl.differential_kinetic_test(org_merge_changed, groupby='condition_celltype')

testing for differential kinetics
    finished (0:30:34) --> added 
    'fit_diff_kinetics', clusters displaying differential kinetics (adata.var)
    'fit_pvals_kinetics', p-values of differential kinetics (adata.var)


In [27]:
scv.tl.velocity(org_merge_changed, mode='dynamical', diff_kinetics=True)
scv.tl.velocity_graph(org_merge_changed)

computing velocities
    finished (0:02:23) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/48 cores)
    finished (0:02:13) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [28]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_changed_velocity_diff_kinetics_genotype_celltype_simplified_stream.pdf"

r_palette = {
    "Neuroepithelium": "#f44336",
    "Dorsal Telencephalic IP": "#8bc34a",
    "Dorsal Telencephalic Neuron": "#009688",
    "Ventral Telencephalic NPC": "#1565c0",
    "Ventral Telencephalic Neuron": "#ffc107",
    "Non-telencephalic NPC": "#ff9800",
    "Non-telencephalic Neuron": "#448aff",
    "Others": "#ad1457"
}

with PdfPages(pdf_path) as pdf:

    scv.pl.velocity_embedding_grid(
        org_merge_changed,
        basis='umap.css.sigPCs',
        color='HNOCA_annot_level_2_pruned_simplified',
        palette=r_palette,
        size=50,
        scale=0.3,               
        alpha=0.5,
        legend_loc='lower left',
        linewidth=1.2,
        legend_fontsize=3,
        show=False
    )

    
    fig = plt.gcf()
    fig.set_size_inches(13, 9)  
    pdf.savefig(fig)           
    plt.close()

computing velocity embedding
    finished (0:00:15) --> added
    'velocity_umap.css.sigPCs', embedded velocity vectors (adata.obsm)


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:65: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return cat(data)  # if data is categorical/array
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype

In [24]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_changed_latent_time_velocity_confidence.pdf"

# scv.tl.latent_time(org_merge_changed)
# scv.tl.velocity_confidence(org_merge_changed)

with PdfPages(pdf_path) as pdf:
    scv.pl.scatter(
        org_merge_changed,  
        color="latent_time", 
        color_map='gnuplot', 
        basis="X_umap.css.sigPCs",
        size=80,
        show=False
    )
    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()
    
    # scv.pl.scatter(
    #     org_merge_changed, 
    #     color="velocity_length", 
    #     color_map="gnuplot", 
    #     basis="X_umap.css.sigPCs",
    #     size=80,
    #     show=False
    # )
    # fig = plt.gcf()
    # fig.set_size_inches(10, 7)
    # pdf.savefig(fig)
    # plt.close()

    # scv.pl.scatter(
    #     org_merge_changed, 
    #     color="velocity_confidence", 
    #     color_map="gnuplot", 
    #     basis="X_umap.css.sigPCs",
    #     size=80,
    #     show=False
    # )

    # fig = plt.gcf()
    # fig.set_size_inches(10, 7)  
    # pdf.savefig(fig)           
    # plt.close()

/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarnin

In [21]:
import scvelo as scv
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Custom palette
r_palette = {
    "Neuroepithelium": "#f44336",
    "Dorsal Telencephalic IP": "#8bc34a",
    "Dorsal Telencephalic Neuron": "#009688",
    "Ventral Telencephalic NPC": "#1565c0",
    "Ventral Telencephalic Neuron": "#ffc107",
    "Non-telencephalic NPC": "#ff9800",
    "Non-telencephalic Neuron": "#448aff",
    "Others": "#ad1457"
}

# Genotypes
genotypes = ['WT', 'KO']

for genotype in genotypes:
    # Subset AnnData for the genotype
    adata_subset = org_merge_changed[org_merge_changed.obs['Genotype'] == genotype].copy()
    
    # PDF path
    pdf_path = f"/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_changed_{genotype}.pdf"
    
    with PdfPages(pdf_path) as pdf:
        # -----------------------------
        # 1️⃣ RNA velocity embedding grid
        # -----------------------------
        scv.pl.velocity_embedding_grid(
            adata_subset,
            basis='umap.css.sigPCs',
            color='HNOCA_annot_level_2_pruned_simplified',
            palette=r_palette,
            size=50,
            scale=0.3,
            alpha=0.5,
            legend_loc='lower left',
            linewidth=1.2,
            legend_fontsize=1,
            show=False
        )
        fig = plt.gcf()
        fig.set_size_inches(13, 9)
        pdf.savefig(fig)
        plt.close()
        
        # -----------------------------
        # 2️⃣ Latent time scatter plot
        # -----------------------------
        scv.pl.scatter(
            adata_subset,  
            color="latent_time", 
            color_map='gnuplot', 
            basis="X_umap.css.sigPCs",
            size=80,
            show=False
        )
        fig = plt.gcf()
        fig.set_size_inches(10, 7)
        pdf.savefig(fig)
        plt.close()


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return isinstance(c, str) and c in data.obs.keys() and cat(data.obs[c])
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:65: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  return cat(data)  # if data is categorical/array
/home/z5358666/.local/lib/python3.10/site-packages/scvelo/plotting/utils.py:68: DeprecationWarning: is_categorical_dtype

In [14]:
# org_merge_D100_WT.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_D100_WT.h5ad")
org_merge_by_D60.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_by_D60.h5ad")
# org_merge_changed_D100.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/251008_D60_D100_merge/org_merge_day100.h5ad")